In [69]:
import numpy as np
import torch
import torch.nn.functional as F
from experiments.exp_filtering_methods import conv2d_diff

lagrange_padding_coefs = [torch.tensor(c, dtype=torch.float32) for c in [
    [[1.0, -3.0, 3.0]],
    [[1.0, -5.0, 10.0, -10.0, 5.0],
     [5.0, -24.0, 45.0, -40.0, 15.0]],
    [[1.0, -7.0, 21.0, -35.0, 35.0, -21.0, 7.0],
     [7.0, -48.0, 140.0, -224.0, 210.0, -112.0, 28.0],
     [28.0, -189.0, 540.0, -840.0, 756.0, -378.0, 84.0]]
]]

def conv2d_lagrange_extrap(image, kernel, returns_padded=False):
    """ extrapolation-based conv2d methods """
    # pad with zero
    k_size = kernel.shape[2]
    p = k_size // 2
    padded = F.pad(image, pad=(p, p, p, p))

    # extrapolation
    # top
    padded[:, :, :p, p:-p] = torch.einsum(
        '...yx,sy->sx', image[:, :, :k_size, :], lagrange_padding_coefs[k_size//2 - 1].flip(dims=(0, 1))
    )
    # bottom
    padded[:, :, -p:, p:-p] = torch.einsum(
        '...yx,sy->sx', image[:, :, -k_size:, :], lagrange_padding_coefs[k_size//2 - 1]
    )
    # left
    padded[:, :, :, :p] = torch.einsum(
        '...yx,sx->ys', padded[:, :, :, p:p+k_size], lagrange_padding_coefs[k_size//2 - 1].flip(dims=(0, 1))
    )
    # right
    padded[:, :, :, -p:] = torch.einsum(
        '...yx,sx->ys', padded[:, :, :, -p-k_size:-p], lagrange_padding_coefs[k_size//2 - 1]
    )
    
    if returns_padded:
        return F.conv2d(padded, kernel), padded
    return F.conv2d(padded, kernel)


M, N, = 5, 5

torch.manual_seed(0)
image = torch.round(torch.randn((1, 1, M, M))*5)
kernel = torch.round(torch.randn((1, 1, N, N))*5)

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

print("Random integer image:")
print(image.numpy())

print("\nRandom integer convolution kernel:")
print(kernel.numpy())

print("\nLagrange polynomial padding:")
lagrange_padded_convolved, lagrange_padded = conv2d_lagrange_extrap(image, kernel, returns_padded=True)
print(lagrange_padded.numpy())
print("Lagrange polynomial padded, then convolved:")
print(lagrange_padded_convolved.numpy())

diff_convolved = conv2d_diff(image, kernel)
print("\nDiff convolved:")
print(diff_convolved.numpy())

print("\nDifference between Lagrange polynomial padded convolved and diff convolved:")
print((diff_convolved - lagrange_padded_convolved).numpy())


Random integer image:
[[[[ -6.  -6.  -1.  -2.   4.]
   [  3.  -2. -11.   2.  -1.]
   [  7.   1.   1.   4.  -1.]
   [ -1.  -3.   6.  10.   0.]
   [  3.  -2.  -4. -12.  -1.]]]]

Random integer convolution kernel:
[[[[-4.  3.  1. -1. -3.]
   [ 5.  2. -3.  0.  6.]
   [ 6. -7. 13. -2.  2.]
   [ 8. 10. -2. -2. -5.]
   [ 6. -1.  4.  2. -4.]]]]

Lagrange polynomial padding:
[[[[17820.  4010.   144.    97.   306.  -230.    50.  5269. 22112.]
   [ 4738.  1049.    18.    13.    81.   -52.    19.  1378.  5788.]
   [  173.    34.    -6.    -6.    -1.    -2.     4.    54.   209.]
   [ -423.   -86.     3.    -2.   -11.     2.    -1.  -122.  -527.]
   [    9.    14.     7.     1.     1.     4.    -1.   -33.  -119.]
   [  135.    35.    -1.    -3.     6.    10.     0.   -26.   -63.]
   [  228.    54.     3.    -2.    -4.   -12.    -1.    88.   348.]
   [ 2168.   524.    74.    34.   -16.  -132.    -6.  1034.  4024.]
   [ 9442.  2274.   298.   153.     4.  -458.   -16.  4073. 16078.]]]]
Lagrange polynom